# 🧠 Clase 7: Sesiones, Estado y Memoria en Google ADK

## 📘 Tutorial Práctico: Convierte a tus agentes en expertos contextuales

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alarcon7a/google-adk-course/blob/main/sources/Clase%207%20-%20Sessions/Sesiones%20y%20Memoria.ipynb)

Bienvenido a la séptima clase del curso. Aquí transformarás agentes básicos en asistentes que recuerdan, personalizan y aprenden de cada usuario. Todo el contenido está en español y alineado con los ejemplos del repositorio.


## 🚀 ¿Qué aprenderás hoy?

- Diferenciar historia de conversación (`events`), estado activo (`state`) y memoria persistente.
- Aplicar prefijos (`user:`, `app:`, `temp:`) para controlar el alcance de la información.
- Inyectar valores del estado directamente en las instrucciones y herramientas de tus agentes.
- Construir un flujo de memoria a largo plazo reutilizando sesiones anteriores con `MemoryService`.


## ⚙️ Configuración inicial

Ejecuta las siguientes celdas si trabajas en un entorno nuevo (Colab, Codespaces o local limpio).


In [ ]:
# Instalación base
!pip install -qU google-adk==1.4.2 litellm==1.73.0 python-dotenv


### 📦 Importaciones y helpers globales

Las utilidades siguientes se reutilizarán en todo el notebook.


In [ ]:
import os

from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService, VertexAiMemoryBankService
from google.adk.tools import load_memory
from google.adk.tools.tool_context import ToolContext
from google.adk.events import Event, EventActions


/home/alarcon7a/anaconda3/envs/env_3_10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


### 🔐 Configura tus credenciales

Necesitas una clave de Google AI Studio (`GOOGLE_API_KEY`) para ejecutar modelos Gemini. Elige una de las opciones siguientes según tu entorno.


In [ ]:
from getpass import getpass

if "GOOGLE_API_KEY" not in os.environ:
    print("🔑 Ingresa tu Google API Key (se guardará solo durante la sesión actual):")
    os.environ["GOOGLE_API_KEY"] = getpass("API Key: ")
else:
    print("✅ API Key detectada en el entorno.")


---


#### 💡 Opción alternativa: archivo `.env`

Si ya guardaste tus claves en un `.env` en la raíz del proyecto, simplemente cárgalas:


In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)
print("🔁 Variables de entorno actualizadas desde .env (si existe).")


🔁 Variables de entorno actualizadas desde .env (si existe).


## 🧭 Panorama general: sesiones vs memoria

- Una **Session** representa una conversación viva entre un usuario y tu agente ADK.
- El atributo **events** guarda todo el historial de turnos, herramientas y callbacks.
- El **state** es un bloc de notas editable que el agente usa dentro de la sesión.
- La **MemoryService** provee conocimientos que sobreviven a múltiples sesiones y usuarios.

Imagina una oficina: los `events` son el acta completa de la reunión, `state` es la libreta en la mesa y `MemoryService` es el archivo de proyectos terminados.


### 🧱 Anatomía de una sesión

- `session.session_id`: identifica el hilo actual.
- `session.state`: diccionario serializable con datos de trabajo.
- `session.events`: lista ordenada de `Event` que documentan cada paso.
- `session.last_update_time`: marca el último cambio, útil para tableros o auditorías.


### 🧭 Prefijos que marcan el alcance

| Prefijo | Alcance | Persistencia* | Uso típico | Ejemplo |
| --- | --- | --- | --- | --- |
| (sin prefijo) | Solo la sesión actual | Depende del `SessionService` | Progreso del flujo, flags temporales | `session.state["paso_actual"] = "confirmar_pago"` |
| `user:` | Todas las sesiones del mismo usuario | Persistente con servicios basados en BD o Vertex | Preferencias y perfil | `session.state["user:idioma"] = "es"` |
| `app:` | Todo el aplicativo y todos los usuarios | Persistente con servicios basados en BD o Vertex | Configuración global, pruebas A/B | `session.state["app:banner_promocional"]` |
| `temp:` | Solo la invocación actual | Nunca se persiste | Cálculos intermedios, datos sensibles de una herramienta | `session.state["temp:raw_reply"]` |

\*`InMemorySessionService` no persiste después de reiniciar, pero respeta el alcance durante la ejecución.


### 🛠️ Helper para ejecutar agentes

Usaremos una función auxiliar para enviar mensajes y capturar la respuesta final del agente.


In [3]:
MODEL = "gemini-2.5-flash"

async def call_agent_async(query: str, runner: Runner, user_id: str, session_id: str) -> str:
    """Envía un mensaje al agente y devuelve la respuesta final de texto."""
    print(f"\n🧑‍💻 Usuario: {query}")
    content = types.Content(role="user", parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final."

    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            break

    print(f"🤖 Agente: {final_response_text}\n")
    return final_response_text


## 📗 Capítulo 1: Personaliza las respuestas con `session.state`

El estado convierte a tu agente en un concierge que recuerda al instante los datos relevantes. A continuación pre-cargamos preferencias y dejamos que el LLM las use a través de la inyección `{clave}`.


In [4]:
APP_NAME = "adk_session_personalizacion"
USER_ID = "cliente_vip"

session_service = InMemorySessionService()

concierge_agent = LlmAgent(
    name="ConciergePersonalizado",
    model=MODEL,
    description="Entrega recomendaciones de viaje basadas en preferencias guardadas.",
    instruction="""Eres un concierge de viajes para clientes premium.
Saluda con calidez y ofrece un plan que combine el tema preferido del cliente.
Nivel del cliente: {user:nivel_membresia}
Tema favorito actual: {tema_favorito}
Usa un tono cercano pero profesional y termina con una pregunta de seguimiento.""",
    output_key="ultima_interaccion"
)

session_id = "sesion_personalizada"

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=session_id,
    state={
        "user:nivel_membresia": "Oro",
        "tema_favorito": "viajes gastronómicos por Asia"
    }
)

runner = Runner(
    agent=concierge_agent,
    app_name=APP_NAME,
    session_service=session_service
)

In [5]:
await call_agent_async(
    query="Hola, quiero inspirarme para mi próximo viaje.",
    runner=runner,
    user_id=USER_ID,
    session_id=session_id
)


🧑‍💻 Usuario: Hola, quiero inspirarme para mi próximo viaje.
🤖 Agente: ¡Qué placer saludarte hoy! Soy tu concierge de viajes y estoy encantado de ayudarte a encontrar la inspiración perfecta para tu próxima aventura.

Sabiendo que eres uno de nuestros valiosos clientes Oro y que tu pasión por los **viajes gastronómicos por Asia** es inigualable, he estado pensando en una experiencia que realmente despierte tus sentidos.

¿Qué te parecería un viaje que combine la elegancia milenaria de **Kioto** con la vibrante energía culinaria de **Singapur**? Podríamos diseñar una ruta que te lleve desde exclusivas ceremonias del té y cenas *kaiseki* con estrellas Michelin en Japón, hasta sesiones privadas de cocina callejera elevada y exploraciones de los *hawker centers* más renombrados en Singapur, pasando por catas de sake artesanales y bares clandestinos con cócteles de autor. Todo esto, por supuesto, con alojamientos de lujo y experiencias personalizadas que solo tú podrías disfrutar.

¿Te suen

'¡Qué placer saludarte hoy! Soy tu concierge de viajes y estoy encantado de ayudarte a encontrar la inspiración perfecta para tu próxima aventura.\n\nSabiendo que eres uno de nuestros valiosos clientes Oro y que tu pasión por los **viajes gastronómicos por Asia** es inigualable, he estado pensando en una experiencia que realmente despierte tus sentidos.\n\n¿Qué te parecería un viaje que combine la elegancia milenaria de **Kioto** con la vibrante energía culinaria de **Singapur**? Podríamos diseñar una ruta que te lleve desde exclusivas ceremonias del té y cenas *kaiseki* con estrellas Michelin en Japón, hasta sesiones privadas de cocina callejera elevada y exploraciones de los *hawker centers* más renombrados en Singapur, pasando por catas de sake artesanales y bares clandestinos con cócteles de autor. Todo esto, por supuesto, con alojamientos de lujo y experiencias personalizadas que solo tú podrías disfrutar.\n\n¿Te suena como el tipo de inspiración que buscas, o tienes alguna otra r

In [6]:
updated_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=session_id
)
print("🧾 Estado después del turno:" )
print(updated_session.state)

🧾 Estado después del turno:
{'user:nivel_membresia': 'Oro', 'tema_favorito': 'viajes gastronómicos por Asia', 'ultima_interaccion': '¡Qué placer saludarte hoy! Soy tu concierge de viajes y estoy encantado de ayudarte a encontrar la inspiración perfecta para tu próxima aventura.\n\nSabiendo que eres uno de nuestros valiosos clientes Oro y que tu pasión por los **viajes gastronómicos por Asia** es inigualable, he estado pensando en una experiencia que realmente despierte tus sentidos.\n\n¿Qué te parecería un viaje que combine la elegancia milenaria de **Kioto** con la vibrante energía culinaria de **Singapur**? Podríamos diseñar una ruta que te lleve desde exclusivas ceremonias del té y cenas *kaiseki* con estrellas Michelin en Japón, hasta sesiones privadas de cocina callejera elevada y exploraciones de los *hawker centers* más renombrados en Singapur, pasando por catas de sake artesanales y bares clandestinos con cócteles de autor. Todo esto, por supuesto, con alojamientos de lujo y ex

### 🔍 Qué sucedió

- `{user:nivel_membresia}` se resolvió antes de enviar la instrucción al modelo.
- El agente guardó el saludo final en `state["ultima_interaccion"]` gracias a `output_key`.
- La próxima vez que consultes el estado podrás reutilizar ese texto, por ejemplo para enviar un resumen por correo.


### 🧪 Extra: Plantillas tolerantes a datos opcionales

El archivo `state.md` describe el sufijo `?` para leer valores opcionales sin romper la ejecución. En este mini flujo comparamos la misma instrucción con y sin la clave `proximo_evento` en el estado.

In [7]:
import uuid

APP_NAME_CONDICIONAL = "adk_state_condicional"
USER_ID_CONDICIONAL = "cliente_condicional"

session_service_condicional = InMemorySessionService()

concierge_condicional = LlmAgent(
    name="ConciergeCondicional",
    model=MODEL,
    instruction="""Saluda al cliente premium y sugiere un plan alineado al tema favorito.
Nivel actual: {user:nivel_membresia}
Tema favorito: {tema_favorito}
Si existe, menciona el contexto adicional {proximo_evento?}.
Cierra con una pregunta corta."""
)

runner_condicional = Runner(
    agent=concierge_condicional,
    app_name=APP_NAME_CONDICIONAL,
    session_service=session_service_condicional
)


In [8]:

async def ejecutar_demo_condicional(state, etiqueta):
    session_id = f"sesion_condicional_{etiqueta}_{uuid.uuid4().hex[:4]}"
    await session_service_condicional.create_session(
        app_name=APP_NAME_CONDICIONAL,
        user_id=USER_ID_CONDICIONAL,
        session_id=session_id,
        state=state
    )
    print(f"▶️ Ejecutando {etiqueta} con sesión {session_id}")
    await call_agent_async(
        query="¿Qué me recomendarías para mi próximo viaje panorámico?",
        runner=runner_condicional,
        user_id=USER_ID_CONDICIONAL,
        session_id=session_id
    )
    session_actualizado = await session_service_condicional.get_session(
        app_name=APP_NAME_CONDICIONAL,
        user_id=USER_ID_CONDICIONAL,
        session_id=session_id
    )
    print("Estado tras la respuesta:", session_actualizado.state, "\n")

In [9]:
await ejecutar_demo_condicional(
    state={
        "user:nivel_membresia": "Platino",
        "tema_favorito": "aventuras en lagos nórdicos"
    },
    etiqueta="sin_contexto_extra"
)

▶️ Ejecutando sin_contexto_extra con sesión sesion_condicional_sin_contexto_extra_2daf

🧑‍💻 Usuario: ¿Qué me recomendarías para mi próximo viaje panorámico?
🤖 Agente: ¡Qué placer saludarle, nuestro/a distinguido/a cliente Platino!

Para su próximo viaje panorámico, y sabiendo su predilección por las **aventuras en lagos nórdicos**, le hemos diseñado una experiencia que captura la esencia de este fascinante tema.

Le proponemos un itinerario que combine la majestuosidad de los paisajes escandinavos con la tranquilidad de sus espejos de agua. Podríamos trazar una ruta que recorra los **fiordos y lagos del oeste de Noruega**, integrando tramos en tren panorámico como el famoso Flåmsbana, que ofrece vistas impresionantes de montañas escarpadas y valles profundos. Luego, se adentraría en las serenas aguas de lagos como el Loen o el Oldevatnet, donde las oportunidades para el remo, la pesca o el senderismo son infinitas. Alternativamente, la **región de los mil lagos en Finlandia** ofrece un

In [10]:
await ejecutar_demo_condicional(
    state={
        "user:nivel_membresia": "Platino",
        "tema_favorito": "aventuras en lagos nórdicos",
        "proximo_evento": "Celebro mi aniversario en junio"
    },
    etiqueta="con_contexto_extra"
)


▶️ Ejecutando con_contexto_extra con sesión sesion_condicional_con_contexto_extra_182f

🧑‍💻 Usuario: ¿Qué me recomendarías para mi próximo viaje panorámico?
🤖 Agente: ¡Estimado cliente Platino!

Es un verdadero placer y un honor ayudarle a planificar su próximo viaje panorámico, especialmente con su pasión por las **aventuras en lagos nórdicos** y la maravillosa ocasión de celebrar su **aniversario en junio**.

Para una ocasión tan significativa y en perfecta sintonía con sus preferencias, le sugeriría un itinerario **personalizado de lujo** que combine la majestuosidad de los fiordos noruegos con la serena belleza de los grandes lagos de Finlandia o Suecia, aprovechando la magia del sol de medianoche en junio.

Imagínese alojándose en **cabañas de diseño exclusivas a orillas del lago**, con saunas privadas y vistas infinitas. Podemos organizar **excursiones privadas en kayak o canoa** con guías expertos para explorar calas secretas, **senderismo guiado** por paisajes vírgenes inmacula

## 📘 Capítulo 2: State como cerebro operativo (herramientas + prefijos)

Cuando las interacciones son largas, conviene que el agente lleve un registro estructurado. El siguiente ejercicio muestra cómo una herramienta actualiza la lista de destinos compartida y cómo el prefijo `temp:` desaparece al terminar el turno.


In [11]:
session_service_plan = InMemorySessionService()
APP_NAME_PLAN = "adk_planificador"
USER_ID_PLAN = "viajero_1"
SESSION_ID_PLAN = "viaje_2025"

def agregar_destino(destino: str, tool_context: ToolContext) -> str:
    """Agrega un destino al itinerario actual y conserva un resumen temporal."""
    historial = list(tool_context.state.get("itinerario", []))
    historial.append(destino)
    tool_context.state["itinerario"] = historial
    tool_context.state["temp:ultimo_destino"] = destino
    print(f"🛠️ Tool: itinerario actualizado -> {historial}")
    print("🛠️ Tool: estado temporal (ultimo_destino) ->", tool_context.state["temp:ultimo_destino"])
    return f"Destino '{destino}' agregado. Pregunta al agente por el itinerario para verlo."

planificador_viajes = LlmAgent(
    name="PlanificadorViajes",
    model=MODEL,
    description="Gestiona un itinerario personalizado actualizando el estado con cada petición.",
    instruction="""Eres un planificador boutique.
Estilo del usuario: {user:estilo}
Usa la herramienta `agregar_destino` siempre que el usuario pida sumar lugares.
Resume el itinerario actual (`{itinerario?}`) antes de despedirte.

planifica de forma creativa y personalizada posibles activdades basadas en el intinerario.
""",
    tools=[agregar_destino]
)

await session_service_plan.create_session(
    app_name=APP_NAME_PLAN,
    user_id=USER_ID_PLAN,
    session_id=SESSION_ID_PLAN,
    state={"user:estilo": "aventurero"}
)

runner_plan = Runner(
    agent=planificador_viajes,
    app_name=APP_NAME_PLAN,
    session_service=session_service_plan
)

In [12]:
await call_agent_async(
    query="Quiero agregar Tokio a mi itinerario y saber qué haríamos allí.",
    runner=runner_plan,
    user_id=USER_ID_PLAN,
    session_id=SESSION_ID_PLAN
)


🧑‍💻 Usuario: Quiero agregar Tokio a mi itinerario y saber qué haríamos allí.


🛠️ Tool: itinerario actualizado -> ['Tokio']
🛠️ Tool: estado temporal (ultimo_destino) -> Tokio
🤖 Agente: ¡Tokio, una elección fantástica para un aventurero como tú! Aquí te dejo algunas ideas para exprimir al máximo la capital japonesa con un toque audaz:

*   **Exploración Urbana Neon:** Sumérgete en el bullicio de Shibuya cruzando su famoso paso de cebra, luego piérdete en las vibrantes calles de Shinjuku, descubriendo izakayas ocultos y la energía de Kabukicho. Para un desafío visual, sube al Tokyo Metropolitan Government Building al atardecer para una vista panorámica impresionante.
*   **Aventura Culinaria Callejera:** Olvídate de los restaurantes turísticos. Atrévete a probar la auténtica comida callejera en Ameya-Yokochō (Ameyoko) en Ueno o en el laberíntico mercado de Tsukiji fuera de las horas punta. Busca yakitori, takoyaki y okonomiyaki en puestos locales.
*   **Reto a la Naturaleza Urbana:** Escapa del cemento subiendo al Monte Takao. No es excesivamente difícil, pero ofre

'¡Tokio, una elección fantástica para un aventurero como tú! Aquí te dejo algunas ideas para exprimir al máximo la capital japonesa con un toque audaz:\n\n*   **Exploración Urbana Neon:** Sumérgete en el bullicio de Shibuya cruzando su famoso paso de cebra, luego piérdete en las vibrantes calles de Shinjuku, descubriendo izakayas ocultos y la energía de Kabukicho. Para un desafío visual, sube al Tokyo Metropolitan Government Building al atardecer para una vista panorámica impresionante.\n*   **Aventura Culinaria Callejera:** Olvídate de los restaurantes turísticos. Atrévete a probar la auténtica comida callejera en Ameya-Yokochō (Ameyoko) en Ueno o en el laberíntico mercado de Tsukiji fuera de las horas punta. Busca yakitori, takoyaki y okonomiyaki en puestos locales.\n*   **Reto a la Naturaleza Urbana:** Escapa del cemento subiendo al Monte Takao. No es excesivamente difícil, pero ofrece rutas de senderismo y vistas espectaculares de la ciudad (y del Monte Fuji en un día claro). ¡Una 

In [13]:
await call_agent_async(
    query="Añade Lisboa y dame una segunda recomendación sorpresiva.",
    runner=runner_plan,
    user_id=USER_ID_PLAN,
    session_id=SESSION_ID_PLAN
)


🧑‍💻 Usuario: Añade Lisboa y dame una segunda recomendación sorpresiva.


🛠️ Tool: itinerario actualizado -> ['Tokio', 'Lisboa']
🛠️ Tool: estado temporal (ultimo_destino) -> Lisboa
🤖 Agente: ¡Excelente elección, Lisboa! Una ciudad que te invita a la aventura con sus colinas, su historia y su vibrante energía. Aquí tienes algunas ideas para una experiencia audaz en la capital portuguesa:

**Para tu aventura en Lisboa:**

*   **Exploración de Colinas y Callejones:** Olvídate del transporte público y prepárate para conquistar las siete colinas de Lisboa a pie. Explora los laberínticos callejones de Alfama, descubre miradores escondidos como el Miradouro da Senhora do Monte para vistas espectaculares, y piérdete en la esencia histórica de la ciudad.
*   **Adrenalina sobre el Tajo:** Alquila una tabla de surf (o toma una clase) y dirígete a las playas cercanas como Carcavelos o Costa da Caparica para desafiar las olas del Atlántico. O para una perspectiva diferente, considera un paseo en kayak o paddleboard por el estuario del Tajo al atardecer.
*   **Sabores Fue

'¡Excelente elección, Lisboa! Una ciudad que te invita a la aventura con sus colinas, su historia y su vibrante energía. Aquí tienes algunas ideas para una experiencia audaz en la capital portuguesa:\n\n**Para tu aventura en Lisboa:**\n\n*   **Exploración de Colinas y Callejones:** Olvídate del transporte público y prepárate para conquistar las siete colinas de Lisboa a pie. Explora los laberínticos callejones de Alfama, descubre miradores escondidos como el Miradouro da Senhora do Monte para vistas espectaculares, y piérdete en la esencia histórica de la ciudad.\n*   **Adrenalina sobre el Tajo:** Alquila una tabla de surf (o toma una clase) y dirígete a las playas cercanas como Carcavelos o Costa da Caparica para desafiar las olas del Atlántico. O para una perspectiva diferente, considera un paseo en kayak o paddleboard por el estuario del Tajo al atardecer.\n*   **Sabores Fuera de Ruta:** Aventúrate en los mercados locales como el Mercado de Campo de Ourique o el Mercado da Ribeira (

In [14]:
await call_agent_async(
    query="Recuérdame cómo va el plan completo.",
    runner=runner_plan,
    user_id=USER_ID_PLAN,
    session_id=SESSION_ID_PLAN
)


🧑‍💻 Usuario: Recuérdame cómo va el plan completo.
🤖 Agente: ¡Claro que sí! Tu plan de viaje actual, lleno de aventuras y contrastes, incluye los siguientes destinos:

*   **Tokio:** Una inmersión en la vibrante cultura urbana, desde el bullicio de Shibuya hasta la serenidad de Yanaka, con toques de adrenalina en Akihabara y retos culinarios.
*   **Lisboa:** Exploración de colinas históricas, desafíos en el Atlántico, una aventura gastronómica auténtica y el descubrimiento de secretos subterráneos.

Además, te he sugerido una recomendación sorpresiva y totalmente aventurera para tu futuro:

*   **Patagonia Chilena:** Una inmersión en la naturaleza salvaje con trekking épico en Torres del Paine, kayak glaciar, cabalgatas gauchas y observación de vida silvestre.

¡Este itinerario promete ser una odisea inolvidable para un viajero como tú!



'¡Claro que sí! Tu plan de viaje actual, lleno de aventuras y contrastes, incluye los siguientes destinos:\n\n*   **Tokio:** Una inmersión en la vibrante cultura urbana, desde el bullicio de Shibuya hasta la serenidad de Yanaka, con toques de adrenalina en Akihabara y retos culinarios.\n*   **Lisboa:** Exploración de colinas históricas, desafíos en el Atlántico, una aventura gastronómica auténtica y el descubrimiento de secretos subterráneos.\n\nAdemás, te he sugerido una recomendación sorpresiva y totalmente aventurera para tu futuro:\n\n*   **Patagonia Chilena:** Una inmersión en la naturaleza salvaje con trekking épico en Torres del Paine, kayak glaciar, cabalgatas gauchas y observación de vida silvestre.\n\n¡Este itinerario promete ser una odisea inolvidable para un viajero como tú!'

In [15]:
state_plan = await session_service_plan.get_session(
    app_name=APP_NAME_PLAN,
    user_id=USER_ID_PLAN,
    session_id=SESSION_ID_PLAN
)
print("🧾 Estado persistente:", state_plan.state)

🧾 Estado persistente: {'user:estilo': 'aventurero', 'itinerario': ['Tokio', 'Lisboa']}


### 🧠 Observaciones clave

- La herramienta actualiza `state["itinerario"]` y usa `temp:ultimo_destino` solo durante la invocación.
- Los prints dentro de la herramienta ayudan a depurar qué datos maneja cada turno.
- El agente combina información `user:` persistente con el progreso de la sesión al responder.


### 🛎️ Caso práctico: mesa de ayuda con SLA activo

En entornos de soporte los agentes deben balancear el SLA comprometido con los avances reales del ticket. Este flujo usa `session.state` para documentar acciones, calcular tiempo consumido y mantener visibles los datos críticos del cliente durante toda la conversación.


In [21]:
session_service_support = InMemorySessionService()
APP_NAME_SOPORTE = "adk_soporte_sla"
USER_ID_SOPORTE = "cliente_andrea"
SESSION_ID_SOPORTE = "ticket_inc_324"

def registrar_avance(accion: str, resultado: str, minutos_estimados: int, tool_context: ToolContext) -> str:
    """Registra avances en el estado del ticket y actualiza el tiempo invertido."""
    historial = list(tool_context.state.get("historial_avances", []))
    avance = {
        "accion": accion,
        "resultado": resultado,
        "minutos": minutos_estimados
    }
    historial.append(avance)
    tool_context.state["historial_avances"] = historial
    tool_context.state["tiempo_total"] = sum(item["minutos"] for item in historial)
    tool_context.state["temp:ultimo_avance"] = resultado
    print("🛠️ Tool: avance registrado ->", avance)
    print("🛠️ Tool: minutos acumulados ->", tool_context.state["tiempo_total"])
    return f"Avance '{accion}' documentado. Tiempo invertido total: {tool_context.state['tiempo_total']} minutos."

analista_soporte = LlmAgent(
    name="AnalistaSoporte",
    model=MODEL,
    description="Gestiona tickets críticos respetando el SLA acordado con el cliente.",
    instruction="""Eres analista de mesa de ayuda.
Ticket: {ticket_id}
Cliente: {user:nombre}
SLA comprometido (min): {sla_minutos}
Tiempo invertido: {tiempo_total?0}

1. Saluda al usuario mencionando cuánto tiempo queda antes de vencer el SLA.
2. Usa la herramienta `registrar_avance` cuando se ejecute una nueva acción o haya que documentar progreso.
3. Mantén una lista clara en `historial_avances` y úsala para resumir al cerrar el ticket.
""",
    tools=[registrar_avance]
)

await session_service_support.create_session(
    app_name=APP_NAME_SOPORTE,
    user_id=USER_ID_SOPORTE,
    session_id=SESSION_ID_SOPORTE,
    state={
        "ticket_id": "INC-324",
        "sla_minutos": 45,
        "user:nombre": "Andrea",
        "historial_avances": [],
        "tiempo_total": 0
    }
)

runner_soporte = Runner(
    agent=analista_soporte,
    app_name=APP_NAME_SOPORTE,
    session_service=session_service_support
)


In [22]:
await call_agent_async(
    query="Hola, ¿hay novedades del ticket? Recuerda que prometimos resolverlo hoy.",
    runner=runner_soporte,
    user_id=USER_ID_SOPORTE,
    session_id=SESSION_ID_SOPORTE
)



🧑‍💻 Usuario: Hola, ¿hay novedades del ticket? Recuerda que prometimos resolverlo hoy.


🤖 Agente: Hola Andrea, gracias por contactarme. Quedan 45 minutos antes de que el SLA de tu ticket INC-324 venza.

¿Hay alguna novedad sobre el ticket o se ha realizado alguna acción que deba registrar?



'Hola Andrea, gracias por contactarme. Quedan 45 minutos antes de que el SLA de tu ticket INC-324 venza.\n\n¿Hay alguna novedad sobre el ticket o se ha realizado alguna acción que deba registrar?'

In [23]:
await call_agent_async(
    query="Registra que reiniciamos el balanceador y que tardaremos 10 minutos en validar.",
    runner=runner_soporte,
    user_id=USER_ID_SOPORTE,
    session_id=SESSION_ID_SOPORTE
)



🧑‍💻 Usuario: Registra que reiniciamos el balanceador y que tardaremos 10 minutos en validar.


🛠️ Tool: avance registrado -> {'accion': 'Reiniciar balanceador', 'resultado': 'Se reinició el balanceador. Pendiente validar en 10 minutos.', 'minutos': 10}
🛠️ Tool: minutos acumulados -> 10
🤖 Agente: Gracias por la información. He registrado que se reinició el balanceador y que la validación tomará 10 minutos.

Ahora mismo el tiempo invertido en el ticket es de 10 minutos, lo que significa que quedan 35 minutos antes de que el SLA venza. Te mantendré informada.



'Gracias por la información. He registrado que se reinició el balanceador y que la validación tomará 10 minutos.\n\nAhora mismo el tiempo invertido en el ticket es de 10 minutos, lo que significa que quedan 35 minutos antes de que el SLA venza. Te mantendré informada.'

In [24]:
await call_agent_async(
    query="Suma otros 10 minutos en testear la solución.",
    runner=runner_soporte,
    user_id=USER_ID_SOPORTE,
    session_id=SESSION_ID_SOPORTE
)



🧑‍💻 Usuario: Suma otros 10 minutos en testear la solución.


🛠️ Tool: avance registrado -> {'accion': 'Testear solución', 'resultado': 'Testeando la solución.', 'minutos': 10}
🛠️ Tool: minutos acumulados -> 20
🤖 Agente: He registrado otros 10 minutos para testear la solución.

El tiempo total invertido en el ticket es ahora de 20 minutos, lo que significa que nos quedan 25 minutos antes de que el SLA venza. Te avisaré en cuanto tengamos más novedades.



'He registrado otros 10 minutos para testear la solución.\n\nEl tiempo total invertido en el ticket es ahora de 20 minutos, lo que significa que nos quedan 25 minutos antes de que el SLA venza. Te avisaré en cuanto tengamos más novedades.'

In [25]:
await call_agent_async(
    query="Acabo de confirmar que volvió el servicio, por favor cierra el ticket y resume lo que hicimos.",
    runner=runner_soporte,
    user_id=USER_ID_SOPORTE,
    session_id=SESSION_ID_SOPORTE
)



🧑‍💻 Usuario: Acabo de confirmar que volvió el servicio, por favor cierra el ticket y resume lo que hicimos.


🤖 Agente: ¡Excelente noticia, Andrea! Me alegra saber que el servicio ha sido restaurado.

Aquí tienes un resumen de las acciones que realizamos para resolver el ticket INC-324:

*   **Reiniciar balanceador**: Se reinició el balanceador y se esperaron 10 minutos para validar.
*   **Testear solución**: Se realizaron pruebas a la solución implementada.

El tiempo total invertido en el ticket fue de 20 minutos, lo que significa que logramos resolverlo con 25 minutos de antelación al vencimiento del SLA.

Procedo a cerrar el ticket. Si tienes alguna otra consulta, no dudes en contactarnos.



'¡Excelente noticia, Andrea! Me alegra saber que el servicio ha sido restaurado.\n\nAquí tienes un resumen de las acciones que realizamos para resolver el ticket INC-324:\n\n*   **Reiniciar balanceador**: Se reinició el balanceador y se esperaron 10 minutos para validar.\n*   **Testear solución**: Se realizaron pruebas a la solución implementada.\n\nEl tiempo total invertido en el ticket fue de 20 minutos, lo que significa que logramos resolverlo con 25 minutos de antelación al vencimiento del SLA.\n\nProcedo a cerrar el ticket. Si tienes alguna otra consulta, no dudes en contactarnos.'

In [26]:
session_soporte_actual = await session_service_support.get_session(
    app_name=APP_NAME_SOPORTE,
    user_id=USER_ID_SOPORTE,
    session_id=SESSION_ID_SOPORTE
)
print("🧾 Estado consolidado del ticket:", session_soporte_actual.state)


🧾 Estado consolidado del ticket: {'ticket_id': 'INC-324', 'sla_minutos': 45, 'user:nombre': 'Andrea', 'historial_avances': [{'accion': 'Reiniciar balanceador', 'resultado': 'Se reinició el balanceador. Pendiente validar en 10 minutos.', 'minutos': 10}, {'accion': 'Testear solución', 'resultado': 'Testeando la solución.', 'minutos': 10}], 'tiempo_total': 20}


### 🧰 Extra: Actualiza múltiples prefijos con `EventActions`

Modificar el estado directo desde el `Session` puede omitir eventos. Con `EventActions.state_delta` controlamos una actualización atómica que mezcla claves de sesión, `user:` y `app:` mientras descartamos las de `temp:` al final de la invocación.

In [33]:
import time
import uuid

APP_NAME_DELTA = "adk_state_delta"
USER_ID_DELTA = "usuario_state_delta"
SESSION_ID_DELTA = f"sesion_delta_{uuid.uuid4().hex[:4]}"

session_service_delta = InMemorySessionService()

session_delta = await session_service_delta.create_session(
    app_name=APP_NAME_DELTA,
    user_id=USER_ID_DELTA,
    session_id=SESSION_ID_DELTA,
    state={
        "task_status": "recopilando",
        "user:login_count": 2
    }
)

acciones_delta = EventActions(
    state_delta={
        "task_status": "validado",
        "user:login_count": session_delta.state.get("user:login_count", 0) + 1,
        "user:last_login_ts": time.time(),
        "app:version_activa": "1.0.2",
        "temp:mensaje_debug": "Validación completada correctamente"
    }
)

evento_delta = Event(
    invocation_id="control_manual",
    author="system",
    actions=acciones_delta,
    timestamp=time.time(),
    content=types.Content(role="system", parts=[types.Part(text="Actualización manual de estado")])
)



In [34]:
session_actualizada = await session_service_delta.get_session(
    app_name=APP_NAME_DELTA,
    user_id=USER_ID_DELTA,
    session_id=SESSION_ID_DELTA
)
print("Estado consolidado:", session_actualizada.state)

Estado consolidado: {'task_status': 'recopilando', 'user:login_count': 2}


In [35]:

await session_service_delta.append_event(session_delta, evento_delta)

session_actualizada = await session_service_delta.get_session(
    app_name=APP_NAME_DELTA,
    user_id=USER_ID_DELTA,
    session_id=SESSION_ID_DELTA
)
print("Estado consolidado:", session_actualizada.state)
print("¿Existe temp:mensaje_debug?:", "temp:mensaje_debug" in session_actualizada.state)

Estado consolidado: {'task_status': 'validado', 'user:login_count': 3, 'user:last_login_ts': 1758383194.6267278, 'app:version_activa': '1.0.2'}
¿Existe temp:mensaje_debug?: False


## 📙 Capítulo 3: Memoria a largo plazo con `MemoryService`

El estado se reinicia con cada sesión, pero la memoria larga permite que el agente aprenda de conversaciones pasadas. Aquí usamos `InMemoryMemoryService` (ideal para prototipos) para almacenar hechos relevantes y recuperarlos en una sesión posterior.


### 🆚 InMemory vs Vertex AI Memory Bank

- **InMemoryMemoryService**: sin configuración adicional, pensado para desarrollo local y pruebas rápidas.
- **VertexAiMemoryBankService**: persistencia administrada por Vertex AI, extracción semántica automática y búsqueda avanzada.
- Selecciona el backend al iniciar tu aplicación (`adk web`, `adk run`) pasando el servicio apropiado al `Runner`.


In [36]:
app_name = "memoria_clase7"
user_id = "memoria_usuario"
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

capturador = LlmAgent(
    name="CapturadorContexto",
    model=MODEL,
    instruction="Agradece al usuario y sintetiza cualquier dato personal o de proyecto que mencione."
)
consultor = LlmAgent(
    name="ConsultorMemoria",
    model=MODEL,
    instruction="""Responde preguntas del usuario. 
Si necesitas recordar detalles pasados, utiliza la herramienta `load_memory` con palabras clave breves y combina los resultados con nueva información.""",
        tools=[load_memory]
    )

runner_captura = Runner(
    agent=capturador,
    app_name=app_name,
    session_service=session_service,
    memory_service=memory_service
)

session_captura = "captura_01"
await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_captura)

Session(id='captura_01', app_name='memoria_clase7', user_id='memoria_usuario', state={}, events=[], last_update_time=1758383273.2661338)

In [37]:

await call_agent_async(
    query="Mi proyecto favorito se llama Atlas y lideré el frente de datos el año pasado.",
    runner=runner_captura,
    user_id=user_id,
    session_id=session_captura
)


🧑‍💻 Usuario: Mi proyecto favorito se llama Atlas y lideré el frente de datos el año pasado.
🤖 Agente: Gracias por compartir. Hemos registrado que tu proyecto favorito es **Atlas** y que lideraste el frente de datos el año pasado.



'Gracias por compartir. Hemos registrado que tu proyecto favorito es **Atlas** y que lideraste el frente de datos el año pasado.'

In [38]:
await call_agent_async(
    query="Mi perro en se llama Rocky y le encanta nadar y encontrar rocas.",
    runner=runner_captura,
    user_id=user_id,
    session_id=session_captura
)



🧑‍💻 Usuario: Mi perro en se llama Rocky y le encanta nadar y encontrar rocas.
🤖 Agente: Gracias por compartir. Hemos registrado que tu perro se llama **Rocky** y que le encanta nadar y encontrar rocas.



'Gracias por compartir. Hemos registrado que tu perro se llama **Rocky** y que le encanta nadar y encontrar rocas.'

In [39]:
completed_session = await session_service.get_session(app_name=app_name, user_id=user_id, session_id=session_captura)
await memory_service.add_session_to_memory(completed_session)
print("📦 Sesión almacenada en la memoria de largo plazo.\n")

📦 Sesión almacenada en la memoria de largo plazo.



In [40]:
runner_consulta = Runner(
    agent=consultor,
    app_name=app_name,
    session_service=session_service,
    memory_service=memory_service
)
session_consulta = "consulta_01"
await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_consulta)

Session(id='consulta_01', app_name='memoria_clase7', user_id='memoria_usuario', state={}, events=[], last_update_time=1758383310.4500537)

In [41]:
await call_agent_async(
    query="¿Recuerdas cuál es mi proyecto favorito y qué rol tuve?",
    runner=runner_consulta,
    user_id=user_id,
    session_id=session_consulta
)


🧑‍💻 Usuario: ¿Recuerdas cuál es mi proyecto favorito y qué rol tuve?


🤖 Agente: Tu proyecto favorito es Atlas y lideraste el frente de datos el año pasado.



'Tu proyecto favorito es Atlas y lideraste el frente de datos el año pasado.'

In [42]:
await call_agent_async(
    query="¿Como se llama mi perro?",
    runner=runner_consulta,
    user_id=user_id,
    session_id=session_consulta
)


🧑‍💻 Usuario: ¿Como se llama mi perro?


🤖 Agente: Tu perro se llama Rocky y le encanta nadar y encontrar rocas.



'Tu perro se llama Rocky y le encanta nadar y encontrar rocas.'

In [43]:
resultados = await memory_service.search_memory(app_name=app_name, user_id=user_id, query="proyecto favorito ")
print("🗂️ Resultados crudos de `search_memory`:", resultados)

🗂️ Resultados crudos de `search_memory`: memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='Mi proyecto favorito se llama Atlas y lideré el frente de datos el año pasado.'
    ),
  ],
  role='user'
), author='user', timestamp='2025-09-20T10:47:55.451440'), MemoryEntry(content=Content(
  parts=[
    Part(
      text='Gracias por compartir. Hemos registrado que tu proyecto favorito es **Atlas** y que lideraste el frente de datos el año pasado.'
    ),
  ],
  role='model'
), author='CapturadorContexto', timestamp='2025-09-20T10:47:55.451736')]


In [46]:
resultados = await memory_service.search_memory(app_name=app_name, user_id=user_id, query="Quien juega con rocas?")
print("🗂️ Resultados crudos de `search_memory`:", resultados)

🗂️ Resultados crudos de `search_memory`: memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='Mi perro en se llama Rocky y le encanta nadar y encontrar rocas.'
    ),
  ],
  role='user'
), author='user', timestamp='2025-09-20T10:48:00.982787'), MemoryEntry(content=Content(
  parts=[
    Part(
      text='Gracias por compartir. Hemos registrado que tu perro se llama **Rocky** y que le encanta nadar y encontrar rocas.'
    ),
  ],
  role='model'
), author='CapturadorContexto', timestamp='2025-09-20T10:48:00.983140')]


### 📌 Lo importante

- `Runner` comparte `session_service` y `memory_service`, por lo que la segunda sesión puede consultar memorias generadas en la primera.
- `memory_service.add_session_to_memory` se ejecuta cuando decides que la conversación contiene información valiosa.
- `load_memory` usa `search_memory` internamente para obtener fragmentos relevantes y pasarlos al LLM.


### 🧠 Extra: Memorias segmentadas por usuario

`memory.md` enfatiza que los recuerdos se aíslan por `user_id`. Este ejemplo crea memorias para dos personas distintas y muestra cómo las búsquedas solo devuelven los datos correspondientes.

In [62]:
import uuid

APP_NAME_MEMORIA_MULTI = "memoria_multi_usuario"
session_service_multi = InMemorySessionService()
memory_service_multi = InMemoryMemoryService()

capturador_perfiles = LlmAgent(
    name="CapturadorPerfiles",
    model=MODEL,
    instruction="""Agradece el mensaje, resume los datos clave (nombre, proyecto, rol) y guarda la respuesta."""
)

runner_perfiles = Runner(
    agent=capturador_perfiles,
    app_name=APP_NAME_MEMORIA_MULTI,
    session_service=session_service_multi,
    memory_service=memory_service_multi
)

async def registrar_perfil(user_id, etiqueta, mensaje):
    session_id = f"{etiqueta}_{uuid.uuid4().hex[:4]}"
    await session_service_multi.create_session(
        app_name=APP_NAME_MEMORIA_MULTI,
        user_id=user_id,
        session_id=session_id
    )
    print(f"▶️ Capturando {etiqueta} en la sesión {session_id}")
    await call_agent_async(
        query=mensaje,
        runner=runner_perfiles,
        user_id=user_id,
        session_id=session_id
    )
    session_completada = await session_service_multi.get_session(
        app_name=APP_NAME_MEMORIA_MULTI,
        user_id=user_id,
        session_id=session_id
    )
    await memory_service_multi.add_session_to_memory(session_completada)

await registrar_perfil(
    user_id="usuario_ana",
    etiqueta="ana_contexto",
    mensaje="Hola, soy Ana y construyo la app Aurora para salud mental. Lidero diseño."
)

await registrar_perfil(
    user_id="usuario_juan",
    etiqueta="juan_contexto",
    mensaje="Soy Juan, trabajo en el proyecto Boreal y estoy en el frente de datos."
)

await registrar_perfil(
    user_id="usuario_ana",
    etiqueta="ana_pasiones",
    mensaje="Disfruto organizar meetups de diseñadores en Bogotá."
)

resultados_ana = await memory_service_multi.search_memory(
    app_name=APP_NAME_MEMORIA_MULTI,
    user_id="usuario_ana",
    query="Aurora o meetups"
)
print("🧾 Recuerdos disponibles para Ana:")
for item in resultados_ana.memories:
    print(item.content.parts[0].text)

resultados_juan = await memory_service_multi.search_memory(
    app_name=APP_NAME_MEMORIA_MULTI,
    user_id="usuario_juan",
    query="Boreal"
)
print("\n🧾 Recuerdos disponibles para Juan:")
for item in resultados_juan.memories:
    print(item.content.parts[0].text)


▶️ Capturando ana_contexto en la sesión ana_contexto_b3fa

🧑‍💻 Usuario: Hola, soy Ana y construyo la app Aurora para salud mental. Lidero diseño.
🤖 Agente: Gracias por el mensaje, Ana. He registrado tus datos clave:

*   **Nombre:** Ana
*   **Proyecto:** App Aurora (salud mental)
*   **Rol:** Líder de diseño

Información guardada.

▶️ Capturando juan_contexto en la sesión juan_contexto_d3b3

🧑‍💻 Usuario: Soy Juan, trabajo en el proyecto Boreal y estoy en el frente de datos.
🤖 Agente: ¡Gracias por el mensaje!

He capturado los siguientes datos clave:
*   **Nombre:** Juan
*   **Proyecto:** Boreal
*   **Rol:** Frente de datos

Información guardada.

▶️ Capturando ana_pasiones en la sesión ana_pasiones_c7c1

🧑‍💻 Usuario: Disfruto organizar meetups de diseñadores en Bogotá.
🤖 Agente: ¡Gracias por el mensaje!

He capturado los siguientes datos:
*   **Nombre:** [No especificado]
*   **Proyecto/Interés:** Meetups de diseñadores en Bogotá
*   **Rol:** Organizador/a

🧾 Recuerdos disponibles para

### ☁️ Usa Vertex AI Memory Bank en producción

1. Habilita Vertex AI en tu proyecto de Google Cloud y crea un Agent Engine con soporte de Memory Bank.
2. Autentícate localmente con `gcloud auth application-default login`.
3. Instancia `VertexAiMemoryBankService(project="TU_PROYECTO", location="us-central1", agent_engine_id="ID_DEL_ENGINE")` y pásala al `Runner`.
4. Mantén la misma lógica de agentes; solo cambia el backend para obtener persistencia, extracción semántica y escalabilidad.


In [ ]:
VERTEX_PROJECT_ID = os.getenv("VERTEX_PROJECT_ID")
VERTEX_LOCATION = os.getenv("VERTEX_LOCATION", "us-central1")
VERTEX_AGENT_ENGINE_ID = os.getenv("VERTEX_AGENT_ENGINE_ID")

if VERTEX_PROJECT_ID and VERTEX_AGENT_ENGINE_ID:
    vertex_memory_service = VertexAiMemoryBankService(
        project=VERTEX_PROJECT_ID,
        location=VERTEX_LOCATION,
        agent_engine_id=VERTEX_AGENT_ENGINE_ID
    )
    print(f"✅ Vertex AI Memory Bank listo: {VERTEX_AGENT_ENGINE_ID} ({VERTEX_LOCATION}).")
else:
    vertex_memory_service = None
    print("⚠️ Define VERTEX_PROJECT_ID y VERTEX_AGENT_ENGINE_ID para habilitar Vertex AI Memory Bank.")


## 🧪 Mejores prácticas

- Mantén el estado pequeño y serializable; evita objetos complejos no soportados.
- Prefiere `output_key`, herramientas y callbacks para actualizar el estado en lugar de modificar `session.state` directamente.
- Documenta qué claves usas y en qué prefijo, de modo que tu equipo pueda extenderlas sin colisiones.
- Envía a memoria solo aquello que será útil en futuras conversaciones; más datos no siempre implican mejores respuestas.


## 📝 Ejercicios sugeridos

1. Implementa un resumen automático por turno que guarde `state["resumen_turno"]` y úsalo en la siguiente respuesta.
2. Añade un callback que bloquee palabras sensibles antes de almacenarlas en memoria.
3. Cambia el ejemplo de memoria para guardar preferencias de playlists musicales y diseña un agente que recomiende canciones.


In [ ]:
# Zona de experimentación 👩‍🔬👨‍🔬
print("Personaliza aquí tus pruebas con session.state y MemoryService.")


## 🎯 Cierre

¡Excelente trabajo! Ahora conoces los bloques que convierten a tus agentes en asistentes verdaderamente contextuales. Sigue explorando combinaciones de estado, callbacks y memoria para llevar tus casos de uso a producción.
